In [61]:
import numpy as np
import pandas as pd
from seqeval.metrics import accuracy_score
import torch.nn as nn
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
from sklearn.model_selection import train_test_split
import re
from transformers import get_linear_schedule_with_warmup
from transformers import DataCollatorForTokenClassification
from transformers import BertForTokenClassification, AdamW
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score,classification_report
from concurrent.futures import ThreadPoolExecutor
from keras_preprocessing.sequence import pad_sequences

In [62]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p) for w, p in zip(s["Mot"].values.tolist(),
                                                           s["Label"].values.tolist())]
        self.grouped = self.data.groupby("sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
        

class Bert_Model(object):
    def __init__(self,sentences=[],labels=[],tag_values=[],tokenizer='bert-base-uncased'):
        self.sentences=sentences
        self.tag_values=tag_values
        self.tag2idx={t: i for i, t in enumerate(self.tag_values)}
        self.tokname=tokenizer
        self.tokenizer=BertTokenizer.from_pretrained(tokenizer,do_lower_case=True)
        self.labels=labels
        self.MAX_LEN=512 #Si le modèle prends trop de temps réduire MAX_LEN, les phrases généralement ne sont pas composées de 512 mots
        #Donc on peut se permettre de le diminuer et pour la prédiction de phrases longues, on split la phrase.
        self.Nlabels=None
        self.model=None

    def getparam(self,MAX_LEN=512):
        self.MAX_LEN=MAX_LEN

    def getTag(self,tag_values):
        self.tag_values=tag_values

    def tokenize_and_preserve_labels(self,sentence,text_labels):
        tokenized_sentence = []
        labels = []

        for word, label in zip(sentence, text_labels):

            tokenized_word = self.tokenizer.tokenize(str(word))
            n_subwords = len(tokenized_word)

            tokenized_sentence.extend(tokenized_word)

            labels.extend([label] * n_subwords)

        return tokenized_sentence, labels
    
    def preprocess(self,random_state=100,test_size=0.1,bs=32,FULL_FINETUNING = True):
        tokenized_texts_and_labels = [self.tokenize_and_preserve_labels(sent, labs) for sent, labs in zip(self.sentences, self.labels) ]
        tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]

        self.Nlabels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

        self.input_ids = pad_sequences([self.tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                                maxlen=self.MAX_LEN, dtype="long", value=0.0,
                                truncating="post", padding="post")

        self.tags = pad_sequences([[self.tag2idx.get(l) for l in lab] for lab in self.Nlabels],
                            maxlen=self.MAX_LEN, value=self.tag2idx["PAD"], padding="post",
                            dtype="long", truncating="post")

        self.attention_masks = [[float(i != 0.0) for i in ii] for ii in self.input_ids]
        tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(self.input_ids, self.tags,
                                                            random_state=random_state, test_size=test_size)
        tr_masks, val_masks, _, _ = train_test_split(self.attention_masks, self.input_ids,
                                             random_state=random_state, test_size=test_size)
        tr_inputs = torch.tensor(tr_inputs)
        val_inputs = torch.tensor(val_inputs)
        tr_tags = torch.tensor(tr_tags)
        val_tags = torch.tensor(val_tags)
        tr_masks = torch.tensor(tr_masks)
        val_masks = torch.tensor(val_masks)
        self.train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
        self.train_sampler = RandomSampler(self.train_data)
        self.train_dataloader = DataLoader(self.train_data, sampler=self.train_sampler, batch_size=bs)

        self.valid_data = TensorDataset(val_inputs, val_masks, val_tags)
        self.valid_sampler = SequentialSampler(self.valid_data)
        self.valid_dataloader = DataLoader(self.valid_data, sampler=self.valid_sampler, batch_size=bs)
        self.model = BertForTokenClassification.from_pretrained(
                self.tokname,
                num_labels=len(self.tag2idx),
                output_attentions = False,
                output_hidden_states = False)
        if FULL_FINETUNING:
            param_optimizer = list(self.model.named_parameters())
            no_decay = ['bias', 'gamma', 'beta']
            optimizer_grouped_parameters = [
                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                'weight_decay_rate': 0.01},
                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                'weight_decay_rate': 0.0}]
        else:
            param_optimizer = list(self.model.classifier.named_parameters())
            optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

        self.optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=3e-5,
            eps=1e-8)
        
    def train_eval(self,epochs=1,max_grad_norm = 1.0,weight=[1,1,1],currloss=np.inf,path='./outputsNSD/modeleNSD'):
        #args.overwrite_output_dir =True

        total_steps = len(self.train_dataloader) * epochs
        class_weights=torch.tensor(weight,dtype=torch.float)
        scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=0,
            num_training_steps=total_steps
        )
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        n_gpu = torch.cuda.device_count()
        loss_values, validation_loss_values = [], []

        for _ in trange(epochs, desc="Epoch"):
            self.model.train()
            total_loss = 0

            for step, batch in enumerate(self.train_dataloader):

                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                self.model.zero_grad()

                outputs = self.model(b_input_ids.long(), token_type_ids=None,
                                attention_mask=b_input_mask, labels=b_labels.long())
                
                loss = outputs[0]
                scores = outputs.logits
                targets = b_labels[:, :scores.shape[1]] 
                loss_fn = nn.CrossEntropyLoss(weight=class_weights)
                loss = loss_fn(scores.view(-1, scores.shape[-1]), targets.view(-1).long())       
                loss.backward()
                
                total_loss += loss.item()

                torch.nn.utils.clip_grad_norm_(parameters=self.model.parameters(), max_norm=max_grad_norm)

                self.optimizer.step()

                scheduler.step()

            
            avg_train_loss = total_loss / len(self.train_dataloader)
            print("Average train loss: {}".format(avg_train_loss))

            loss_values.append(avg_train_loss)

            #Evaluation du modèle
            self.model.eval()
            
            eval_loss, eval_accuracy = 0, 0
            nb_eval_steps, nb_eval_examples = 0, 0
            predictions , true_labels = [], []
            for batch in self.valid_dataloader:
                batch = tuple(t.to(device) for t in batch)
                b_input_ids, b_input_mask, b_labels = batch

                with torch.no_grad():

                    outputs = self.model(b_input_ids.long(), token_type_ids=None,
                                    attention_mask=b_input_mask, labels=b_labels.long())
                
                logits = outputs[1].detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                
                eval_loss += outputs[0].mean().item()
                predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
                true_labels.extend(label_ids)

            eval_loss = eval_loss / len(self.valid_dataloader)
            validation_loss_values.append(eval_loss)
            #print(len(predictions),len(true_labels))
            print("Validation loss: {}".format(eval_loss))
            pred_tags = [self.tag_values[p_i] for p, l in zip(predictions, true_labels)
                                        for p_i, l_i in zip(p, l) if self.tag_values[l_i] != "PAD"]
            valid_tags = [self.tag_values[l_i] for l in true_labels
                                        for l_i in l if self.tag_values[l_i] != "PAD"]
            print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
            print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags,average='weighted')))
            classification_rep = classification_report(valid_tags, pred_tags)
            print("Classification Report:\n", classification_rep)
            print()
            #self.model.save_pretrained('./outputs/modeleFT')
            self.loss_values=loss_values
            self.validation_loss_values=validation_loss_values
            if currloss>eval_loss:
                self.save_model(path)
                print("Model saved successfully.")
                currloss=eval_loss

    def save_model(self, path):
            self.model.save_pretrained(path)
            self.tokenizer.save_pretrained(path)

    def load_model(self, path):
            self.model = BertForTokenClassification.from_pretrained(path)
            self.tokenizer = BertTokenizer.from_pretrained(path)
            self.model.eval()

    def predict(self, sentence):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        torch.cuda.empty_cache()
        #self.model.cuda()
        self.model.eval()
        tokenized_sentence = self.tokenizer.tokenize(sentence)
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)
        tokens_tensor = torch.tensor([indexed_tokens]).to(device)
        attention_mask = [[float(i != 0.0) for i in tokens_tensor[0]]]
        attention_mask = torch.tensor(attention_mask).to(device)
        with torch.no_grad():
            outputs = self.model(tokens_tensor, token_type_ids=None, attention_mask=attention_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_indices = np.argmax(logits, axis=2)[0]
        labels = [self.tag_values[label_idx] for label_idx in label_indices]
        return list(zip(tokenized_sentence, labels))
    

    def batch_predict(self, sentences):
        with ThreadPoolExecutor() as executor:
            predictions = list(executor.map(self.predict, sentences))
        return predictions    


                
    def plotloss(self):
        sns.set(style='darkgrid')

        sns.set(font_scale=1.5)
        plt.rcParams["figure.figsize"] = (12,6)

        plt.plot(self.loss_values, 'b-o', label="training loss")
        plt.plot(self.validation_loss_values, 'r-o', label="validation loss")

        plt.title("Learning curve")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()

        plt.show()

def convert2cue(oui,sentence):
    """ Applique le modèle de cue detection au phrases d'entrée pour transformer les mots labellisés cue en [CUE]"""
    new_sentence=[]
    for sen in sentence:
        test_sentence=[x.lower() for x in sen]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        n_gpu = torch.cuda.device_count()
        tokenized_sentence = oui.tokenizer.encode(test_sentence)
        input_ids = torch.tensor([tokenized_sentence])
        with torch.no_grad():
            output = oui.model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

        tokens = oui.tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(oui.tag_values[label_idx])
                new_tokens.append(token)
        #print(list(map(lambda x, y: '[CUE]' if x == 'CUE' else y, new_labels[1:-1], test_sentence)))
        new_sentence.append(list(map(lambda x, y: '[CUE]' if x == 'CUE' else y, new_labels[1:-1], test_sentence)))
    return new_sentence

In [74]:
model.tag_values

{'B-Mood': 0,
 'O': 1,
 'B-Drug': 2,
 'I-Procedure': 3,
 'B-Condition': 4,
 'B-Person': 5,
 'I-Condition': 6,
 'B-Procedure': 7,
 'B-Observation': 8,
 'I-Drug': 9,
 'I-Observation': 10,
 'I-Person': 11,
 'I-Mood': 12,
 'PAD': 13}

In [63]:
df_train = pd.read_csv('./data/train.csv').head(5000)
df = pd.read_csv('../data/clini_data.csv')

/tmp/ipykernel_30351/2102769662.py:2: DtypeWarning: Columns (2,3,4,5,10,11,12,13,14,15,17,18,19,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,37,42,43,68,79,80,81,82,88,89,94,95,97,98,99,100,101,102,104,105,109,110,111,112,113,114,115,116,118,121,122,151,163,169,184,192,193,194,195,196,200,206,209,210,212,213,214,215,216,217,218,219,220,222,223,225,226,228,229,230,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,262,263,264,265,266,275,279,280,282,283,284,285,286,287,300,301,302,303,304,308,320,321,322,324,325,326) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/clini_data.csv')


In [64]:
getter = SentenceGetter(df_train)

In [65]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
sentences[0]

['With',
 'severe',
 'comorbidities',
 ',',
 'such',
 'as',
 'cardiovascular',
 'disease',
 ',',
 'chronic',
 'obstructive',
 'pulmonary',
 'disease',
 ',',
 'diabetes',
 'mellitus',
 ',',
 'and',
 'chronic',
 'renal',
 'dysfunction',
 '.']

In [66]:
labels = [[s[1] for s in sentence] for sentence in getter.sentences]
print(labels[0])

['O', 'O', 'B-Condition', 'O', 'O', 'O', 'B-Condition', 'I-Condition', 'O', 'B-Condition', 'I-Condition', 'I-Condition', 'I-Condition', 'O', 'B-Condition', 'I-Condition', 'O', 'O', 'B-Condition', 'I-Condition', 'I-Condition', 'O']


In [67]:
tag_values = list(set(df_train["Label"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
tag2idx

{'B-Mood': 0,
 'O': 1,
 'B-Drug': 2,
 'I-Procedure': 3,
 'B-Condition': 4,
 'B-Person': 5,
 'I-Condition': 6,
 'B-Procedure': 7,
 'B-Observation': 8,
 'I-Drug': 9,
 'I-Observation': 10,
 'I-Person': 11,
 'I-Mood': 12,
 'PAD': 13}

In [75]:
model = Bert_Model(sentences=sentences, labels=labels, tag_values=tag_values)

In [76]:
model.getparam(100)

In [77]:
model.preprocess()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/onyxia/work/.venv/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [71]:
[1]*len(tag2idx)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [78]:
model.train_eval(weight=[1]*len(tag2idx))

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Average train loss: 1.2004535396893818
Validation loss: 0.6252216100692749
Validation Accuracy: 0.0016155088852988692
Validation F1-Score: 8.138583805032086e-06
Classification Report:
                precision    recall  f1-score   support

  B-Condition       0.00      0.00      0.00        59
       B-Drug       0.00      0.00      0.00        25
       B-Mood       0.00      0.00      0.00         9
B-Observation       0.00      0.00      0.00         1
     B-Person       0.00      0.00      0.00         9
  B-Procedure       0.00      0.00      0.00        11
  I-Condition       0.00      0.00      0.00        71
       I-Drug       0.00      0.00      0.00        10
       I-Mood       0.00      0.00      0.00         5
     I-Person       0.00      0.00      0.00         4
  I-Procedure       0.00      0.00      0.00        19
            O       1.00      0.00      0.01       396
          PAD       0.00      0.00      0.00         0

     accuracy                           0.0

/home/onyxia/work/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onyxia/work/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onyxia/work/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/onyxia/work/.venv/lib/pytho

Model saved successfully.
